In [25]:
import pandas as pd
import spotipy 
from spotipy.oauth2 import SpotifyOAuth
import configparser
from difflib import SequenceMatcher # For similarity of strings
import time

In [2]:
parser = configparser.ConfigParser()

parser.read('credentials.conf')
username = parser.get('spotify_config', 'username')
client_id = parser.get('spotify_config', 'client_id')
client_secret = parser.get('spotify_config', 'client_secret')

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri='http://localhost:8080/',
    scope ='playlist-modify-private,playlist-modify,playlist-read-private'))

In [4]:
playlists = sp.user_playlists(username)

playlist_names = [item['name'] for item in playlists['items']]
playlist_ids = [item['id'] for item in playlists['items']]
names_to_ids = dict(zip(playlist_names, playlist_ids))

new_playlist = 'YOASOBI-yt'

if new_playlist in playlist_names:
    chosen_playlist_id = names_to_ids[new_playlist]
else:
    created_playlist = sp.user_playlist_create(username, name=new_playlist, public=False)
    chosen_playlist_id = created_playlist['id']

In [5]:
df = pd.read_csv('data/yt_titles.csv')

In [16]:
def similar(name1, name2):
    ''' 
    TODO documentation
    '''
    return SequenceMatcher(None, name1, name2).ratio()

In [23]:
def choose_track(track1, track2, song_name):
    ''' 
    TODO documentation
    '''
    track1_similarity = similar(song_name, track1)
    track2_similarity = similar(song_name, track2)

    if track1_similarity >= track2_similarity:
        return 0
    return 1

In [24]:
songs_to_add = []

for i in range(df.shape[0]):
    song_name = df['video_title'][i]

    potential_tracks = sp.search(q=f'{song_name}', type='track', market = 'CA')
    track_index = choose_track(
        track1=potential_tracks['tracks']['items'][0]['name'],
        track2=potential_tracks['tracks']['items'][1]['name'],
        song_name=song_name
    )

    songs_to_add.append(potential_tracks['tracks']['items'][track_index]['id'])
    time.sleep(3)

In [26]:
songs_to_add

['7ydiBXSeyMcdDdvXM89VLE',
 '4BE1OloRc9xwjyqA4wFFuN',
 '3dPtXHP0oXQ4HCWHsOA9js',
 '0T4AitQuq8IJhWBWuZwkFA',
 '3QXl80080U9z3pmMDWaYLv',
 '0loZ1KfQSLJxYR0Y7dImKN',
 '5hVAQTNarrb3lmWqg9Ens4',
 '19fhOFi6pNGeZe5uiFlm7c',
 '2YbNZLoiREBYZo4HeKB8Np',
 '6EuubEs5qOmJdpNhkU47OZ',
 '2YbRWFyb8RiQNsd7uQ91Tz']